In [58]:
setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/bulk/hahn_2023/cortex")

library(dplyr)
library(data.table)

source("/mnt/lareaulab/reliscu/code/FindModules/FindModules.R")

Allowing multi-threading with up to 48 threads.


Here I run FM to (hopefully) find modules representing each of the cell types present in the single-cell data

In [ ]:
expr_tpm <- fread("hahn_2023_cortex_STAR_TPM_SampleNetworks/All_06-23-55/hahn_2023_cortex_STAR_TPM_All_68_outliers_removed.csv", data.table=FALSE)

# For duplicate genes, choose row with highest mean expression

mean_expr <- data.frame(
    Index=1:nrow(expr_tpm), 
    Gene=expr_tpm[,1], 
    Expr=rowMeans(expr_tpm[,-1])
)

mean_expr <- mean_expr %>%
    group_by(Gene) %>%
    slice_max(Expr)

print(dim(mean_expr))

expr_tpm <- expr_tpm[mean_expr$Index,]

[1] 52633     3


In [ ]:
# Subset to genes in the top X percentile of mean expression

prob <- .6
mean_expr <- rowMeans(expr_tpm[,-1])
print(sum(mean_expr >= quantile(mean_expr, prob)))
expr_tpm_filtered <- expr_tpm[mean_expr >= prob,]

# Log transform data
# expr_tpm_log2 <- log2(expr_tpm_filtered[,-1] + 1)

expr <- data.frame(Gene=expr_tpm_filtered[,1], expr_tpm_filtered[,-1])

[1] 21053


In [51]:
# Order samples by covariates of interest

sampleinfo <- read.csv("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/bulk/hahn_2023/cortex/hahn_2023_cortex_sampleinfo.csv")
sampleinfo <- sampleinfo %>%
    arrange(age, sex_y)
sampleinfo <- sampleinfo[sampleinfo$Run %in% colnames(expr),]
    
expr <- expr[, c(1, match(sampleinfo$Run, colnames(expr)[-1]) + 1)]
all.equal(sampleinfo$Run, colnames(expr)[-1])

[1] TRUE

In [52]:
samplegroups <- as.factor(sampleinfo$age)
merge.param <- 0.85

In [53]:
# Encountering errors when calculating distance in FM; problem seems related to some genes having 0 MAD (median absolute deviation)

# Filtering these in advance:

X <- as.matrix(expr[,-1])
mads <- apply(X, 1, mad, na.rm = TRUE)
bad  <- which(!is.finite(mads) | mads == 0)

length(bad)
head(expr[,1][bad])

mask <- mads > 0
expr <- expr[mask,]

[1] 597

[1] "Cga"     "Gh"      "Gm11192" "Gm11594" "Gm12571" "Gm12719"

In [47]:
expr[1:4,1:4]

,Gene,SRR21355373,SRR21355377,SRR21355364
,<chr>,<dbl>,<dbl>,<dbl>
3941,0610009E02Rik,2.200651,3.060461,2.407408
34041,0610009L18Rik,6.536998,6.141230,7.392501
32471,0610010K14Rik,2.547075,1.828318,2.717159
17310,0610030E20Rik,8.497339,7.914483,9.126372


In [ ]:
FindModules(
  projectname="hahn_2023_cortex_STAR_TPM_OR_mergeParam0.85",
  expr=expr,
  geneinfo=1,
  sampleindex=2:ncol(expr),
  samplegroups=samplegroups,
  subset=NULL,
  simMat=NULL,
  saveSimMat=FALSE,
  simType="Bicor",
  beta=1,
  overlapType="None",
  TOtype="signed",
  TOdenom="min",
  MIestimator="mi.mm",
  MIdisc="equalfreq",
  signumType="rel",
  iterate=TRUE,
  signumvec=c(.999,.99,.98,.97,.96,.85), # signumvec=c(.99999, .9999, .999, .99, .98, .97, .96, .95),
  minsizevec=c(4,6,8,10,12,15), # minsizevec=c(3,4,5,6,8,10),
  signum=NULL,
  minSize=NULL,
  merge.by="ME",
  merge.param=merge.param,
  export.merge.comp=T,
  ZNCcut=2,
  calcSW=FALSE,
  loadTree=FALSE,
  writeKME=TRUE,
  calcBigModStat=FALSE,
  writeModSnap=TRUE
)

In [ ]:
# Debugging

#   projectname="hahn_2023_cortex_STAR_TPM_SN_QN_mergeParam0.85"
#   expr=expr
#   geneinfo=1
#   sampleindex=2:ncol(expr)
#   samplegroups=samplegroups
#   subset=NULL
#   simMat=NULL
#   saveSimMat=FALSE
#   simType="Bicor"
#   beta=1
#   overlapType="None"
#   TOtype="signed"
#   TOdenom="min"
#   MIestimator="mi.mm"
#   MIdisc="equalfreq"
#   signumType="rel"
#   iterate=TRUE
#   signumvec=c(.999, .99, .98, .97, .96, .85) # signumvec=c(.99999 .9999 .999 .99 .98 .97 .96 .95)
#   minsizevec=c(4, 6, 8, 10, 12, 15) # minsizevec=c(3456810)
#   signum=NULL
#   minSize=NULL
#   merge.by="ME"
#   merge.param=merge.param
#   export.merge.comp=T
#   ZNCcut=2
#   calcSW=FALSE
#   loadTree=FALSE
#   writeKME=TRUE
#   calcBigModStat=FALSE
#   writeModSnap=TRUE
#   varCut=NULL

# 	if(length(unique(expr[,1]))!=length(expr[,1])){
		
# 		stop("Identifiers in column 1 must be unique!")
		
# 	}
	
# ## Create root directory, if necessary:
	
# 	if(length(grep(paste(projectname,"_Modules",sep=""),getwd()))==1){
		
# 		breakdir=strsplit(getwd(),split="/")
# 		BNroot=c(1:length(breakdir[[1]]))[is.element(breakdir[[1]],paste(projectname,"_Modules",sep=""))]
# 		setwd(paste(breakdir[[1]][1:BNroot],collapse="/"))
		
# 	} else {
		
# 		dir.create(paste(projectname,"_Modules",sep=""))
# 		setwd(paste(getwd(),"/",projectname,"_Modules",sep=""))
		
# 	}
	
# 	BNrootDir=getwd()	

# ## Check for missing data and set NA = 0 per user (recommended for RNAseq data):

# 	missing.values=sum(is.na(expr[,sampleindex]))
# 	total.values=nrow(expr)*length(sampleindex)
	
# 	if(missing.values>0){
# 	  cat("\n")
# 		print(paste(signif(missing.values/total.values*100,2),"% of data are missing. Setting NA = 0 (recommended for RNAseq data)",sep=""),quote=F)
# 		cat("\n")
# 		expr[,sampleindex][is.na(expr[,sampleindex])]=0
# 	}
	
# 	## Check for values <= 0, and scale so minimum values = 1 (avoids issues with log tranformations)
	
# 	zero.values=sum(expr[,sampleindex]<=0)
	
# 	if(zero.values>0){
	  
# 	  cat("\n")
# 	  print("Gene expression values <= 0 are present. Scaling all data so minimum expression = 1",quote=F)
# 	  cat("\n")
# 	  expr[,sampleindex]=expr[,sampleindex]+abs(min(expr[,sampleindex],na.rm=T))+1
	  
# 	} 
	
# ## Note: FM 0.88 excludes genes with low variance from kME table:

# 	if(is.null(varCut)){
# 	  varCut=0
# 	}
	
# 	datExprall=t(expr[,sampleindex])
# 	colnames(datExprall)=as.character(expr[,1])
# 	var.all=apply(datExprall,2,var)
# 	datExprall=datExprall[,var.all>varCut] 
	
# 	if(sum(var.all<varCut)>0){
		
# 		cat("\n")
# 		print(paste("Note: ",sum(var.all<varCut)," features had 0 variance and were excluded",sep=""))
# 		cat("\n")
# 		ExcludedFeatures=expr[var.all<varCut,geneinfo]
# 		write.table(ExcludedFeatures,file="Excluded_genes_with_0_variance.csv",sep=",",row.names=F,col.names=T)
		
# 	}
		
# 	if(!is.null(subset)){
		
# 		subset=as.logical(subset)
# 		datExpr1=t(expr[subset,sampleindex])
# 		colnames(datExpr1)=as.character(expr[subset,1])
		
# 	} else {
		
# 		datExpr1=t(expr[,sampleindex])
# 		colnames(datExpr1)=as.character(expr[,1])
		
# 	}
	
# 	var1=apply(datExpr1,2,var)
	
# 	if(sum(var1<varCut)>0){
		
# 		datExpr1=datExpr1[,var1>varCut]
		
# 	}
	
# 	if(is.null(samplegroups)){
		
# 		samplegroups=c(rep(1,length(sampleindex)))
		
# 	} else {
	  
# 	  if(sum(is.na(samplegroups))>0){
	    
# 	    samplegroups=as.character(samplegroups)
# 	    samplegroups[is.na(samplegroups)]="None"
	    
# 	  }
		
# 		if(is.character(samplegroups)){
			
# 		  samplegroups <- as.factor(samplegroups)
			
# 		}
		
# 	}

In [ ]:
# simMat=bicor(datExpr1,use="p")
# collectGarbage()
# diag(simMat)=0
# collectGarbage()


Warning message in bicor(datExpr1, use = "p"):
“bicor: zero MAD in variable 'x'. Pearson correlation was used for individual columns with zero (or missing) MAD.”


In [ ]:
# print("Testing parameters...")
# 		cat("\n")
		
# 		if(signumType=="rel"){
		
# 			if(((dim(datExpr1)[2]*(dim(datExpr1)[2]-1))/2)>1e+06){
			
# 				relsignumvec=quantile(sample(simMat,1000000,replace=FALSE),probs=signumvec)
# 				collectGarbage()
			
# 			} else {
		
# 				relsignumvec=quantile(simMat[upper.tri(simMat)],probs=signumvec)
# 				collectGarbage()
			
# 			}
			
# 		}

[1] "Testing parameters..."



ERROR: Error in quantile.default(sample(simMat, 1e+06, replace = FALSE), probs = signumvec): missing values and NaN's not allowed if 'na.rm' is FALSE
